In [2]:
import geopandas as gpd
import rasterio
from rasterio.features import rasterize
from shapely.geometry import mapping
import numpy as np
from PIL import Image
import os
import logging


In [16]:
input_geojson_path = "/Users/mischabauckhage/Documents/ETH/02_Master/3_Semester/GMP2/gmp2/01_Segmentation/output/old_national_1975_skeleton.geojson"
output_tiff_path = "/Users/mischabauckhage/Documents/ETH/02_Master/3_Semester/GMP2/gmp2/02_DEM/output/height_map_old_national_1975.png"
output_png_path = "output/"
height_attribute = "height"
input_crs = "EPSG:2056"
resolution = 0.5

In [40]:
import numpy as np
import rasterio
from rasterio.features import rasterize
from shapely.geometry import mapping
import geopandas as gpd

# Load the GeoDataFrame
gdf = gpd.read_file(input_geojson_path)



# Select only the 'height' attribute and the geometry
gdf = gdf[['height', 'geometry']]
# Check validity of each geometry
gdf['is_valid'] = gdf['geometry'].is_valid


# Show all invalid geometries
invalid_geometries = gdf[~gdf['is_valid']]
print(invalid_geometries)

# Remove all features that have 'is_valid' == False
gdf = gdf[gdf['is_valid']]

gdf.head()




     height geometry  is_valid
265     530     None     False
338     540     None     False


,height,geometry,is_valid
0,520,"LINESTRING (2.5822e+06 1.1952e+06, 2.5822e+06 ...",True
1,540,"LINESTRING (2.5817e+06 1.1949e+06, 2.5817e+06 ...",True
2,550,"LINESTRING (2.5819e+06 1.194e+06, 2.5819e+06 1...",True
3,550,"LINESTRING (2.5816e+06 1.194e+06, 2.5816e+06 1...",True
4,560,"LINESTRING (2.5816e+06 1.1938e+06, 2.5816e+06 ...",True


In [41]:
#gdf = gdf.to_crs(epsg=2056)

# Verify if all geometries are now valid
print(gdf['is_valid'].all())  # Should return True if all geometries are valid


# Get the total bounds of the geometries
minx, miny, maxx, maxy = gdf.total_bounds
print(minx, miny, maxx, maxy)

gdf.head()

True
2581552.698868844 1192587.1014954355 2585425.9010940734 1195404.8796298308


,height,geometry,is_valid
0,520,"LINESTRING (2.5822e+06 1.1952e+06, 2.5822e+06 ...",True
1,540,"LINESTRING (2.5817e+06 1.1949e+06, 2.5817e+06 ...",True
2,550,"LINESTRING (2.5819e+06 1.194e+06, 2.5819e+06 1...",True
3,550,"LINESTRING (2.5816e+06 1.194e+06, 2.5816e+06 1...",True
4,560,"LINESTRING (2.5816e+06 1.1938e+06, 2.5816e+06 ...",True


In [32]:

# Define the output raster dimensions (based on the resolution)

width = int((maxx - minx) / resolution)
height = int((maxy - miny) / resolution)

# Create an affine transform for the raster
transform = rasterio.transform.from_bounds(minx, miny, maxx, maxy, width, height)

# Create an empty array to store the raster data
raster = np.zeros((height, width), dtype=rasterio.float32)



# Prepare geometries and corresponding 'hoehe' values for rasterization
shapes = ((mapping(geom), value) for geom, value in zip(gdf.geometry, gdf[height_attribute]))

# Rasterize the geometries using the 'hoehe' values
raster = rasterize(
    shapes=shapes,
    out_shape=raster.shape,
    transform=transform,
    fill=0,  # Background value
    dtype=rasterio.float32
)

# At this point, 'raster' contains the rasterized data
